# EnergyPlus Parameterization and Sensitivity Analysis for Residential Single Family Homes


## Daniel Xu, Keller Lab

This notebook was created in order to document the workflow and processes of EnergyPlus within Python. This is for my project involving the parameterization of EnergyPlus. Given that EnergyPlus has over 2000 available inputs, more research needs to be done in order to document which parameters are most significant and relevant in home energy modeling and simulations. 

The goal of this project is to run EnergyPlus on a model single family residential home. Some of the inputs will be fixed to default variables. Other variables, such as window thickness, thermostat set-points, etc. will be varied on pre-defined distributions. 

The distributions will be defined as seen in this paper: 
https://www.sciencedirect.com/science/article/abs/pii/S037877881631372X?via=ihub

### Importing necessary libraries

In [ ]:
from eppy import modeleditor
from eppy.modeleditor import IDF
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

### Changing the working directory 

In [ ]:
os.chdir("/jumbo/keller-lab/projects/eplus_sensitivity/scripts")
print("Current working directory:", os.getcwd())

### Setting EnergyPlus Input Data Dictionary

In [ ]:
iddfile = "/jumbo/keller-lab/Applications/EnergyPlus-24-1-0/Energy+.idd"
IDF.setiddname(iddfile)

### Setting a Skeleton IDF File 

This modifiable IDF file is provided by EnergyPlus in its initial download. It provides a mock residential home in Chicago with multiple zones. I will assume that some of the inputs are fixed and others are varied on a distribution. 

Directly from the EnergyPlus documentation: 

"This file does the basic test of an air distribution system in a residential home. A two speed heat pump with a supplmental gas heater provides space heating and cooling. It provides ventilation through the ZoneAirBalance:OutdoorAir model."

In [ ]:
idfname = "/jumbo/keller-lab/Applications/EnergyPlus-24-1-0/ExampleFiles/SingleFamilyHouse_TwoSpeed_MultiStageElectricSuppCoil.idf"

preidf = IDF(idfname)

# preidf.printidf()

print(preidf.idfobjects['Site:Location'])

### Weather Station Data Retrieval for EnergyPlus Simulations

This project aims to enhance the accuracy of building energy simulations by sourcing precise local weather data. By leveraging the NCEI (National Centers for Environmental Information) API, we are able to identify and retrieve data from the nearest weather station to any specified location. This localized weather data is crucial for feeding into EnergyPlus simulations, ensuring that our building models operate under realistic environmental conditions. The accurate simulation of energy usage and needs based on actual weather data helps in designing more efficient and sustainable buildings.

For more detailed information on how to utilize the NCEI API, including obtaining access tokens and making API requests, please visit the NCEI Web Services Documentation.

In [ ]:
import requests
from datetime import datetime, timedelta

def get_active_weather_stations(api_key, lat, lon):
    base_url = 'https://www.ncei.noaa.gov/cdo-web/api/v2/stations'
    headers = {'token': api_key}
    # Define the date one year ago from today
    one_year_ago = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')

    # Set the request to retrieve more stations for a broader check
    params = {
        'extent': f'{lat-0.05},{lon-0.05},{lat+0.05},{lon+0.05}',
        'limit': '1000',  # Adjust limit as needed
        'sortfield': 'mindate',
        'sortorder': 'desc'
    }

    response = requests.get(base_url, headers=headers, params=params)
    
    if response.status_code != 200:
        print(f"Failed to fetch data: {response.status_code} - {response.text}")
        return []

    try:
        data = response.json()
        active_stations = []
        # Filter stations by checking if the 'maxdate' is within the last year
        for station in data.get('results', []):
            if station['maxdate'] >= one_year_ago:
                active_stations.append(station)
        return active_stations
    except ValueError:
        print("Failed to decode JSON from response.")
        return []

api_key = 'SrgpVmvZhbtZXRSdBgknhaRSQlhTNzBt'
latitude = 41.78  
longitude = -87.75  

active_stations = get_active_weather_stations(api_key, latitude, longitude)
print(f"Found {len(active_stations)} active stations:")
for station in active_stations:
    print(station)


Based on this location, we will then use the diyepw package to create time series weather data. More information about diyepw can be found at: 

https://diyepw.readthedocs.io/en/latest/tutorial.html (PNNL)

In [ ]:
import diyepw

diyepw.create_amy_epw_files_for_years_and_wmos(
  [2020,2021,2022,2023],
  [725340],
  max_records_to_interpolate=6,
  max_records_to_impute=48,
  max_missing_amy_rows=5,
  allow_downloads=True,
  amy_epw_dir='./'
)

### Editing IDF parameters 



In [ ]:
def update_people_per_area(idf_file_path, idd_file_path):
    # Load the EnergyPlus IDD file
    IDF.setiddname(idd_file_path)

    # Load the IDF file
    idf = IDF(idf_file_path)

    # Retrieve all PEOPLE objects
    people_objects = idf.idfobjects['PEOPLE']

    # Define the range for the uniform distribution
    low = 0.002
    high = 0.060

    # Find the specific PEOPLE object by name and update it
    for person in people_objects:
        if person.Name == 'LIVING ZONE People':
            # Generate a new value for People per Floor Area
            new_people_per_area = np.random.uniform(low, high)
            
            # Update the People per Zone Floor Area
            person.People_per_Floor_Area = new_people_per_area
            print(f"Updated {person.Name} with People per Floor Area = {new_people_per_area}")

            # Construct a new file path for the modified file
            dir_path, file_name = os.path.split(idf_file_path)
            new_file_path = os.path.join(dir_path, 'modified_' + file_name)
            
            # Save the changes
            idf.save(new_file_path)
            print(f"File saved to {new_file_path}")
            break
    else:
        print("No PEOPLE object found with the name 'LIVING ZONE People'")

# Paths to your IDF and IDD files
idf_file_path = "/jumbo/keller-lab/Applications/EnergyPlus-24-1-0/ExampleFiles/SingleFamilyHouse_TwoSpeed_MultiStageElectricSuppCoil.idf"
idd_file_path = "/jumbo/keller-lab/Applications/EnergyPlus-24-1-0/Energy+.idd"

# Execute the function
update_people_per_area(idf_file_path, idd_file_path)
